In [1]:
import requests
from bs4 import BeautifulSoup
import os
import polars as pl
from zipfile import ZipFile

import source
from extractions import (
    cvmDownloadFiles,
    cvmReadFromLanding,
    cvmWriteFiles,
    cvmReadFiles
)


In [3]:
#cvmDownloadFiles('itr')

Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2011.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2012.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2013.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2014.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2015.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2016.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2017.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2018.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2019.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2020.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2021.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2022.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2023.zip


In [2]:
df = cvmReadFiles('raw', 'dre')
df.schema

OrderedDict([('CNPJ_CIA', Utf8),
             ('DT_REFER', Date),
             ('VERSAO', Int64),
             ('DENOM_CIA', Utf8),
             ('CD_CVM', Int64),
             ('GRUPO_DFP', Utf8),
             ('MOEDA', Utf8),
             ('ESCALA_MOEDA', Utf8),
             ('ORDEM_EXERC', Utf8),
             ('DT_INI_EXERC', Date),
             ('DT_FIM_EXERC', Date),
             ('CD_CONTA', Utf8),
             ('DS_CONTA', Utf8),
             ('VL_CONTA', Float64),
             ('ST_CONTA_FIXA', Utf8)])